In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
td = []
with open("data/time_diffs_no_overwrite.txt", "r") as f:
    for line in f:
        td.append(float(line.strip()))
td = np.array(td[1:])
print(td[:10])

[]


In [3]:
print(td.sum())
print(td.mean())
print(td.var())
print(td.std())
print(td.min())
print(td.max())
plt.plot(td[:100])

0.0
nan
nan
nan


/opt/anaconda3/envs/meng/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice.
  
/opt/anaconda3/envs/meng/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/envs/meng/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Degrees of freedom <= 0 for slice
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/meng/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/anaconda3/envs/meng/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/envs/meng/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
# how often does this clip?
# if i+1 takes more time than i, then we clip

In [ ]:
# min of 0.0029 is a speedup of 5.73x!
# first thing: try to normalize the fastest playback
# let's clip to 2x speedup 1/60/2 = 0.00833
td2x = td.clip(1/60/2, None)

In [ ]:
print(td2x.sum())
print(td2x.mean())
print(td2x.var())
print(td2x.std())
print(td2x.min())
print(td2x.max())
plt.plot(td2x[:100])

In [ ]:
# i need to be able to draw a line that is smooth and stays above the points
# this seems like the best we can do in a buffer size 1 case
td_diffs = td.copy()
scale = 0.95
for i in range(1, td_diffs.shape[0]):
    if td[i] > td_diffs[i-1]:
        td_diffs[i] = td[i]
        scale = 0.99
    else:
        td_diffs[i] = max(td[i], td_diffs[i-1]*scale)
        scale *= 0.99
td_diffs = np.array(td_diffs)

In [ ]:
print(td_diffs.sum())
print(td_diffs.mean())
print(td_diffs.var())
print(td_diffs.std())
print(td_diffs.min())
print(td_diffs.max())
ind = 100
plt.plot(td_diffs[:ind])
plt.plot(td[:ind])

In [ ]:
# how often does this clip?
td_c_diff = [td_diffs[0]]
clipping = [int(td_diffs[0] < td[0])]
for i in range(1, td_batches.shape[0]):
    td_c_diff.append(td_c_diff[i-1] + max(td[i], td_diffs[i]))
    clipping.append(int(td_diffs[i] < td[i]))
td_c_diff = np.array(td_c_diff)
clipping = np.array(clipping)

In [ ]:
print(td_c_diff[-1])
plt.plot(td_c_diff)

In [ ]:
print(clipping.sum())
plt.plot(clipping)

In [ ]:
# now how can we increase the buffer size to handle outliers better?
td_batches = []
buffer_size = 128
for i in range(td.shape[0]-buffer_size):
    td_batches.append(td[i:i+buffer_size].sum())
td_batches = np.array(td_batches) / buffer_size * (td.shape[0] / (td.shape[0] - buffer_size))

In [ ]:
print(td_batches.sum())
print(td_batches.mean())
print(td_batches.var())
print(td_batches.std())
print(td_batches.min())
print(td_batches.max())
ind = 100
plt.plot(td_batches[:ind])
plt.plot(td[:ind])

In [ ]:
# for each batch, if i play the average then i get the same result as above
# danger is using a running average
# but can i approximate this away as well?
# in this simulation, when would i get gaps?
td_c_batch = [td_batches[0]]
clipping = [int(td_batches[0] < td[0])]
for i in range(1, td_batches.shape[0]):
    td_c_batch.append(td_c_batch[i-1] + max(td[i], td_batches[i]))
    clipping.append(int(td_batches[i] < td[i]))
td_c_batch = np.array(td_c_batch)
clipping = np.array(clipping)

In [ ]:
print(td_c_batch[-1])

plt.plot(td_c_batch)

In [ ]:
print(clipping.sum())
plt.plot(clipping)

In [ ]:
# the gaps are forming because we always need 10 items in the buffer
# what if the buffer size is flexible?